In [1]:
%load_ext autoreload
%autoreload 2

# Create 400 neuron dataset

In [2]:
import os

import colorcet as cc
import matplotlib.pyplot as plt
import numpy as np
import torch
from analysis.data_gen_utils import (
    all_units_except,
    combine_datasets,
    download_IBL,
    extract_IBL,
    make_dataset,
)
from analysis.projections import learn_manifold_umap, pca, pca_train
from ceed.models.ceed import CEED
from mpl_toolkits.mplot3d import Axes3D

2023-11-14 20:30:41.399692: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 20:30:41.399761: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 20:30:41.402111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 20:30:41.600953: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 20:30:42.940867: W tensorflow/compiler/

In [5]:
pid_sess1 = 'dab512bd-a02d-4c1f-8dbc-9155a163efc0'
pid_sess2 = 'febb430e-2d50-4f83-87a0-b5ffbb9a4943'
save_folder_sess1 = "/media/cat/data/IBL_data_CEED/dab512bd-a02d-4c1f-8dbc-9155a163efc0"
save_folder_sess2 = "/media/cat/data/IBL_data_CEED/febb430e-2d50-4f83-87a0-b5ffbb9a4943"
t_window = [0, 1199]  # in seconds
overwrite = False
rec1, meta_file_sess1 = download_IBL(
    pid=pid_sess1, t_window=t_window, save_folder=save_folder_sess1, overwrite=overwrite
)
rec2, meta_file_sess2 = download_IBL(
    pid=pid_sess2, t_window=t_window, save_folder=save_folder_sess2, overwrite=overwrite
)

INFO:one.api:cache over 76 day(s) old
INFO:one.api:No newer cache available
Downloading: /home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/cache/ap/danlab/Subjects/DY_016/2020-09-12/001/raw_ephys_data/probe00/chunk_000000_to_001198/_spikeglx_ephysData_g0_t0.imec0.ap.8cb6749e-846b-4c1e-94da-2d026c410921.cbin Bytes: 10974282242


 25%|█████▋                 | 2580.0/10465.891115188599 [01:39<05:03, 26.01it/s]

KeyboardInterrupt



In [6]:
"""extract the all data needed to make CEED dataset
spike_idx_sess: spike_times, channels, neurons (if use_labels=True)
geom_sess: channels x 2
chan_idx_sess: waveform extraction channels for each channel
templates_sess: templates across all channels for all neurons
"""
recompute = True

if recompute:
    spike_idx_sess1, geom_sess1, chan_idx_sess1, templates_sess1 = extract_IBL(
        rec=rec1,
        meta_fp=meta_file_sess1,
        pid=pid_sess1,
        t_window=t_window,
        use_labels=True,
    )
    spike_idx_sess2, geom_sess2, chan_idx_sess2, templates_sess2 = extract_IBL(
        rec=rec2,
        meta_fp=meta_file_sess2,
        pid=pid_sess2,
        t_window=t_window,
        use_labels=True,
    )
    np.save("spike_idx_sess1.npy", spike_idx_sess1)
    np.save("geom_sess1.npy", geom_sess1)
    np.save("chan_idx_sess1.npy", chan_idx_sess1)
    np.save("templates_sess1.npy", templates_sess1)
    np.save("spike_idx_sess2.npy", spike_idx_sess2)
    np.save("geom_sess2.npy", geom_sess2)
    np.save("chan_idx_sess2.npy", chan_idx_sess2)
    np.save("templates_sess2.npy", templates_sess2)
else:
    spike_idx_sess1 = np.load("spike_idx_sess1.npy")
    geom_sess1 = np.load("geom_sess1.npy")
    chan_idx_sess1 = np.load("chan_idx_sess1.npy")
    templates_sess1 = np.load("templates_sess1.npy")
    spike_idx_sess2 = np.load("spike_idx_sess2.npy")
    geom_sess2 = np.load("geom_sess2.npy")
    chan_idx_sess2 = np.load("chan_idx_sess2.npy")
    templates_sess2 = np.load("templates_sess2.npy")

/media/cat/cole/miniforge-pypy3/envs/ceed/lib/python3.10/site-packages/ibllib/atlas/atlas.py:13: DeprecationWarning: ibllib.atlas.atlas.AllenAtlas is deprecated. Use iblatlas.atlas.AllenAtlas instead
  warnings.warn(warning_text, DeprecationWarning)


INFO:one.api:Downloading from AWS


/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0
/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0
/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0
/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0

INFO:one.api:Downloading from AWS



/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0
/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0
/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0
/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0

...

INFO:one.api:Downloading from AWS



/home/cole/Downloads/ONE/openalyx.internationalbrainlab.org/danlab/Subjects/DY_0


5 inactive units


Align and get templates:   0%|          | 0/461 [00:00<?, ?it/s]

Cleaned templates:   0%|          | 0/461 [00:00<?, ?it/s]

In [8]:
dy016_unit_ids

'/home/cole/CEED/notebooks/400neuron_unit_ids/dy016_unit_ids.npy'

In [9]:
# session DYO16 units to get data from and dataset save path
dy016_unit_ids_path = os.path.join(
    os.getcwd(), "400neuron_unit_ids", "dy016_unit_ids.npy"
)
selected_units_sess1 = np.load(dy016_unit_ids_path)
dataset_folder_sess1 = save_folder_sess1 + "/ds"

# make first dataset for training with DY016 units
# will create a folder with the spike, probe channel number, and corresponding channel location datasets in the train, val, test splits
# optionally also saves out spatial and temporal noise covariance matrices
inference = False
train_num = 200
val_num = 0
test_num = 200
save_covs = False
num_chans_extract = 21
normalize = False  # True for cell-type dataset
shift = False
save_fewer = False
(
    train_set1,
    val_set1,
    test_set1,
    train_geom_locs1,
    val_geom_locs1,
    test_geom_locs1,
    train_max_chan1,
    val_max_chan1,
    test_max_chan1,
) = make_dataset(
    rec=rec1,
    spike_index=spike_idx_sess1,
    geom=geom_sess1,
    save_path=dataset_folder_sess1,
    chan_index=chan_idx_sess1,
    templates=templates_sess1,
    unit_ids=selected_units_sess1,
    train_num=train_num,
    val_num=val_num,
    test_num=test_num,
    save_covs=save_covs,
    num_chans_extract=num_chans_extract,
    normalize=normalize,
    shift=shift,
    inference=inference,
    save_fewer=save_fewer,
)

copying over noise covariances from previous recording from /home/cole/CEED/analysis/noise_covariances_IBL_dy016


200it [04:57,  1.49s/it]


making train, val, test splits
saving split results


In [10]:
# session DYO16 units to get data from and dataset save path
dy009_unit_ids_path = os.path.join(
    os.getcwd(), "400neuron_unit_ids", "dy009_unit_ids.npy"
)
selected_units_sess2 = np.load(dy009_unit_ids_path)
dataset_folder_sess2 = save_folder_sess2 + "/ds"

# make second dataset for training with DY009 units
# will create a folder with the spike, probe channel number, and corresponding channel location datasets in the train, val, test splits
# optionally also saves out spatial and temporal noise covariance matrices
inference = False
train_num = 200
val_num = 0
test_num = 200
save_covs = False
num_chans_extract = 21
normalize = False  # True for cell-type dataset
shift = False
save_fewer = False
(
    train_set2,
    val_set2,
    test_set2,
    train_geom_locs2,
    val_geom_locs2,
    test_geom_locs2,
    train_max_chan2,
    val_max_chan2,
    test_max_chan2,
) = make_dataset(
    rec=rec2,
    spike_index=spike_idx_sess2,
    geom=geom_sess2,
    save_path=dataset_folder_sess2,
    chan_index=chan_idx_sess2,
    templates=templates_sess2,
    unit_ids=selected_units_sess2,
    train_num=train_num,
    val_num=val_num,
    test_num=test_num,
    save_covs=save_covs,
    num_chans_extract=num_chans_extract,
    normalize=normalize,
    shift=shift,
    inference=inference,
    save_fewer=save_fewer,
)

copying over noise covariances from previous recording from /home/cole/CEED/analysis/noise_covariances_IBL_dy016


200it [05:14,  1.57s/it]


making train, val, test splits
saving split results


In [18]:
combined_ds_path = "/media/cat/data/IBL_data_CEED/400neuron_200spike_ds"

# combine the two training datasets into a larger one for more unit diversity
dataset_list = [dataset_folder_sess1, dataset_folder_sess2]
combine_datasets(dataset_list, combined_ds_path)

In [25]:
data_dir = combined_ds_path
exp_name = "spikesorting_CEED_400n_paper_experiment"
log_dir = data_dir + "/logs/"
ckpt_dir = data_dir + "/saved_models/"
batch_size = 512
num_extra_chans = 5  # 11 channels total
save_metrics = True
epochs = 400
aug_p_dict = {
    "collide": 0.4,
    "crop_shift": 0.4,
    "amp_jitter": 0.5,
    "temporal_jitter": 0.7,
    "smart_noise": (0.5, 1.0),
}
# subsample the 10 neuron dataset used in the paper from the 400 neurons
# will output the results on training metrics
testing_unit_ids = [11, 13, 16, 69, 84, 89, 277, 267, 332, 343]
test_units = testing_unit_ids

print(test_units)
# Train the 400 neuron, 200 spike, 11 channel model benchmarked in the supplement of the paper
# (very similar results to the 1200 spike version)
ceed_test = CEED(num_extra_chans=5)
ceed_test.train(
    data_dir=data_dir,
    exp_name=exp_name,
    log_dir=log_dir,
    epochs=epochs,
    ckpt_dir=ckpt_dir,
    batch_size=batch_size,
    save_metrics=save_metrics,
    aug_p_dict=aug_p_dict,
    units_list=test_units,
)

[11, 13, 16, 69, 84, 89, 277, 267, 332, 343]
Using projector; batchnorm False with depth 3; hidden_dim=512
True spikes_train.npy
Units: 10
[  6   7   9  11  13  16  17  20  21  22  23  25  27  28  29  31  32  33
  35  36  37  40  41  43  45  47  48  49  53  56  64  67  69  70  71  72
  73  75  77  78  80  81  84  88  89  91  92  95  96  97  98 100 101 102
 103 104 105 106 107 109 110 112 113 114 116 119 120 126 128 129 132 133
 134 135 136 137 138 142 143 148 149 152 156 157 161 162 164 169 170 171
 178 179 182 183 185 186 187 188 189 190 191 192 195 204 206 207 212 213
 214 215 216 217 220 223 225 226 231 232 233 237 238 239 241 244 245 250
 252 255 256 265 267 270 271 274 277 278 280 281 289 290 291 292 294 297
 298 300 304 305 306 307 312 320 321 325 326 328 330 331 332 333 334 337
 340 343 351 353 355 357 359 362 365 367 369 374 378 381 386 388 395 398
 402 410 411 412 413 414 415 416 422 423 427 430 434 451 474 476 479 481
 485 503 504 505 507 508 509 510 511 512 513 515 516 518 5

TypeError: save_reps() got an unexpected keyword argument 'rep_after_proj'

In [8]:
data_dir = '/media/cat/data/IBL_data_CEED/400neuron_200spike_ds'
ceed_test = CEED(num_extra_chans=5)
ceed_test.load('/media/cat/data/IBL_data_CEED/400neuron_200spike_ds/saved_models/spikesorting_CEED_400n_paper_experiment/')
fc_transformed_inference_data, fc_inference_labels = ceed_test.load_and_transform(
    data_dir=data_dir,units_list=test_units, file_split="test"
)

Using projector; batchnorm False with depth 3; hidden_dim=512
loading from previous checkpoint:  /media/cat/data/IBL_data_CEED/400neuron_200spike_ds/saved_models/spikesorting_CEED_400n_paper_experiment/
Units: 10
[  6   7   9  11  13  16  17  20  21  22  23  25  27  28  29  31  32  33
  35  36  37  40  41  43  45  47  48  49  53  56  64  67  69  70  71  72
  73  75  77  78  80  81  84  88  89  91  92  95  96  97  98 100 101 102
 103 104 105 106 107 109 110 112 113 114 116 119 120 126 128 129 132 133
 134 135 136 137 138 142 143 148 149 152 156 157 161 162 164 169 170 171
 178 179 182 183 185 186 187 188 189 190 191 192 195 204 206 207 212 213
 214 215 216 217 220 223 225 226 231 232 233 237 238 239 241 244 245 250
 252 255 256 265 267 270 271 274 277 278 280 281 289 290 291 292 294 297
 298 300 304 305 306 307 312 320 321 325 326 328 330 331 332 333 334 337
 340 343 351 353 355 357 359 362 365 367 369 374 378 381 386 388 395 398
 402 410 411 412 413 414 415 416 422 423 427 430 434 451 

In [18]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.mixture import GaussianMixture

covariance_type = "full"
n_clusters = 10
reps_train = fc_transformed_inference_data
reps_test = fc_transformed_inference_data
scores = []
for i in range(100):
    gmm = GaussianMixture(n_clusters, random_state=i, covariance_type=covariance_type).fit(
        reps_test
    )
    gmm_cont_test_labels = gmm.predict(reps_test)
    score = adjusted_rand_score(fc_inference_labels, gmm_cont_test_labels) * 100
    scores.append(score)
    print(f"num_comps: {fc_transformed_inference_data.shape[1]}, rand_score: {score}")

num_comps: 5, rand_score: 72.33670150553434
num_comps: 5, rand_score: 71.2848922944476
num_comps: 5, rand_score: 70.5123699496437
num_comps: 5, rand_score: 79.19715003445258
num_comps: 5, rand_score: 69.560607717423
num_comps: 5, rand_score: 73.97441944819067
num_comps: 5, rand_score: 64.08665674483946
num_comps: 5, rand_score: 79.19715003445258
num_comps: 5, rand_score: 70.91720378592922
num_comps: 5, rand_score: 69.64250532908505
num_comps: 5, rand_score: 69.52792002428603
num_comps: 5, rand_score: 64.46958500564186
num_comps: 5, rand_score: 79.19715003445258
num_comps: 5, rand_score: 63.633082437487374
num_comps: 5, rand_score: 64.92950868695104
num_comps: 5, rand_score: 70.46614853755234
num_comps: 5, rand_score: 72.75924835699175
num_comps: 5, rand_score: 79.19715003445258
num_comps: 5, rand_score: 65.08267346865011
num_comps: 5, rand_score: 79.19715003445258
num_comps: 5, rand_score: 70.5123699496437
num_comps: 5, rand_score: 79.19715003445258
num_comps: 5, rand_score: 79.1971500